In [1]:
import pandas as pd
import os
import csv

import math
import numpy as np
import matplotlib.pyplot as plt

读取训练集和测试集的csv文件

In [2]:
train_data = pd.read_csv('./airfoil_noise_samples.csv')
test_data = pd.read_csv('./airfoil_noise_test.csv')
train_data.head()

,Frequency,Angle,Displacement,Chord length,Velocity,Thickness,Sound Pressure
0,2175.611424,15.138865,21.075119,0.088194,66.764401,0.044771,122.365215
1,2962.923620,13.400893,13.200889,0.048462,78.221903,0.011041,129.296236
2,4430.810843,2.164599,13.959536,0.226743,57.053201,0.011499,121.827380
3,4939.695645,13.857682,18.203793,0.021705,23.896377,0.021475,114.998132
4,2193.979785,9.298757,11.007713,0.052737,38.917034,0.001741,125.639641


定义特征和目标变量

In [3]:
X_train = train_data.iloc[:, :-1]
Y_train = train_data.iloc[:, -1]
X_test = test_data.iloc[:, :-1]
Y_test = test_data.iloc[:, -1]

数据归一化，防止数据过拟合

In [4]:
#归一化
X_train = (X_train - X_train.mean()) / X_train.std()
X_test = (X_test - X_test.mean()) / X_test.std()
X_train.head()

,Frequency,Angle,Displacement,Chord length,Velocity,Thickness
0,-0.275324,1.240614,1.602819,-0.558959,0.934190,2.181587
1,0.007620,0.928480,-0.074895,-0.989410,1.631190,-0.232215
2,0.535150,-1.089521,0.086745,0.942044,0.343425,-0.199430
3,0.718034,1.010518,0.991043,-1.279290,-1.673619,0.514487
4,-0.268723,0.191750,-0.542182,-0.943098,-0.759861,-0.897771


使用knn训练

In [5]:
# knn回归
def knn_regression(X_train, Y_train, X_test, k):
    y_pred = []
    for x_test in X_test:
        distances = np.linalg.norm(X_train - X_test, axis=1)
        k_neighbors_indices = np.argsort(distances)[:k]
        y_pred.append(np.mean(Y_train[k_neighbors_indices]))
    return np.array(y_pred)

# 3. 交叉验证找到最佳k值
def cross_validation(X, y, k_values, num_folds=5):
    best_k = None
    best_mse = float('inf')
    fold_size = len(X) // num_folds
    
    for k in k_values:
        mse_values = []
        for fold in range(num_folds):
            start = fold * fold_size
            end = (fold + 1) * fold_size
            X_val = X[start:end]
            y_val = y[start:end]
            X_train = np.concatenate([X[:start], X[end:]])
            y_train = np.concatenate([y[:start], y[end:]])
            y_pred = knn_regression(X_train, y_train, X_val, k)
            mse = np.mean((y_pred - y_val) ** 2)
            mse_values.append(mse)
        
        mean_mse = np.mean(mse_values)
        if mean_mse < best_mse:
            best_mse = mean_mse
            best_k = k
    
    return best_k

# 4. 使用5折交叉验证选择最佳k值
k_values = list(range(1, 21))  # 尝试k值从1到20
best_k = cross_validation(X_train, Y_train, k_values, num_folds=5)

# 测试
y_pred_test = knn_regression(X_train, Y_train, X_test, best_k)
mse_test = np.mean((y_pred_test - Y_test) ** 2)
print("样本外MSE:", mse_test)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/yjj/workspace/jupyter_venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_3781/3282784969.py", line 38, in <module>
    best_k = cross_validation(X_train, Y_train, k_values, num_folds=5)
  File "/tmp/ipykernel_3781/3282784969.py", line 25, in cross_validation
    y_pred = knn_regression(X_train, y_train, X_val, k)
  File "/tmp/ipykernel_3781/3282784969.py", line 5, in knn_regression
    distances = np.linalg.norm(X_train - X_test, axis=1)
  File "/home/yjj/workspace/jupyter_venv/lib/python3.10/site-packages/pandas/core/generic.py", line 2102, in __array_ufunc__
    return arraylike.array_ufunc(self, ufunc, method, *inputs, **kwargs)
  File "/home/yjj/workspace/jupyter_venv/lib/python3.10/site-packages/pandas/core/arraylike.py", line 273, in array_ufunc
    result = maybe_dispatch_ufunc_to_dunder_op(self, ufunc, method, *in